# برای اینکه رفرنس تابع مان هنگام استفاده از دکوریتور درست باشد از دکوریتور functools.wraps(func)استفاده میکنیم
#برای اینکه ممکن است تابع ورودی داشته باشد یا ممکن است نداشته باشد در دکوریتور از *args ,**kwargsاستفاده می کنیم

In [11]:
%%writefile my_decorator.py

import functools 
from datetime import datetime

def my_decorator(func):
    @functools.wraps(func)
    def not_during_night(*args , **kwargs):
            if 7<=datetime.now().hour<20:
                func(*args , **kwargs)
                return func(*args , **kwargs)
            else:
                return'this is out of time'

    return not_during_night

Overwriting my_decorator.py


In [12]:
from my_decorator import my_decorator
@my_decorator
def wrapper_during_morning(name):
    print(f'good morning {name}')

In [13]:
wrapper_during_morning('ziba')

In [14]:
@my_decorator
def say_whee():
    print('whee')

In [15]:
say_whee()

In [16]:
say_whee

<function __main__.say_whee()>

In [9]:
import functools , time


def timer(func):
    """"print the runtimes of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args , **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time-start_time
        print(f'finished {func.__name__!r} in {run_time:.4f} secs')
        return value
    return wrapper_timer


@timer
def waste_time(num_time):
    for _ in range (num_time):
        return sum ([i**2 for i in range (10000)])

In [10]:
waste_time(4)

finished 'waste_time' in 0.0011 secs


333283335000

In [11]:
waste_time

<function __main__.waste_time(num_time)>

In [12]:
import functools
def debug(func):
    """print the function signature and return value """
    def wrapper_debug(*args,**kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f'{k}={v!r}' for k , v in kwargs.items()]
        arguments = args_repr + kwargs_repr
        signature = ' , '.join(arguments)
        
        print(f'calling {func.__name__} ({signature})')
        value = func(*args,**kwargs)
        print(f'{func.__name__!r} returned {value!r} ')
        return value
    return wrapper_debug
        

In [13]:
@debug
def make_greeting(name , age=None):
    if age is None:
        return f'Howdy {name}'
    else:
        return f'whoa {name}! {age} already , you are growing up'

In [14]:
make_greeting('ziba' , 58)

calling make_greeting ('ziba' , 58)
'make_greeting' returned 'whoa ziba! 58 already , you are growing up' 


'whoa ziba! 58 already , you are growing up'

In [15]:
import math
math.factorial= debug(math.factorial)

def aproximate_e(terms=18):
    return sum(1/math.factorial(n) for n in range(terms))

In [16]:
aproximate_e()

calling factorial (0)
'factorial' returned 1 
calling factorial (1)
'factorial' returned 1 
calling factorial (2)
'factorial' returned 2 
calling factorial (3)
'factorial' returned 6 
calling factorial (4)
'factorial' returned 24 
calling factorial (5)
'factorial' returned 120 
calling factorial (6)
'factorial' returned 720 
calling factorial (7)
'factorial' returned 5040 
calling factorial (8)
'factorial' returned 40320 
calling factorial (9)
'factorial' returned 362880 
calling factorial (10)
'factorial' returned 3628800 
calling factorial (11)
'factorial' returned 39916800 
calling factorial (12)
'factorial' returned 479001600 
calling factorial (13)
'factorial' returned 6227020800 
calling factorial (14)
'factorial' returned 87178291200 
calling factorial (15)
'factorial' returned 1307674368000 
calling factorial (16)
'factorial' returned 20922789888000 
calling factorial (17)
'factorial' returned 355687428096000 


2.7182818284590455

In [17]:
import functools , time

def slow_down(func):
    """sleep one second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args , **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def count_down(num):
    if num<1:
        print('liftoff')
        
    else:
        print(num)
        count_down(num-1)
        
    

In [18]:
count_down(5)

5
4
3
2
1
liftoff


# اگر دکوراتور را قبل از کلاس بنویسیم دکوراتور روی متد initاثر می کند

In [19]:
#@debug
class Time_waster():
    @debug
    def __init__(self,max_num):
        self.max_num=max_num
     
    @timer
    def waste_time(self,num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(num_times)])

In [20]:
t=Time_waster(3)

calling __init__ (<__main__.Time_waster object at 0x0000026218527550> , 3)
'__init__' returned None 


In [21]:
t.waste_time(400)

finished 'waste_time' in 0.0118 secs


# دکوراتور با ارگمان ورودی 

In [22]:
import functools

def repeat(num_repeat):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args,**kwargs):
            for _ in range(num_repeat):
                value=func(*args,**kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

In [23]:
@repeat(num_repeat=3)
def greet(name):
    print(f'hi {name}')

In [24]:
greet('ziba')

hi ziba
hi ziba
hi ziba


## nested_decorator

# در دکوریتور های تو در تو ابتدا دکوریتوری که به تابع چسبیده و نزدیک تر است اعمال می شود بعد دکوریتورهای بالایی

In [25]:
@debug
@repeat(5)
@slow_down
def greet(name):
    print(f'hi {name}')

In [26]:
greet('mohammad')

calling greet ('mohammad')
hi mohammad
hi mohammad
hi mohammad
hi mohammad
hi mohammad
'greet' returned None 


## stateful decorator

In [13]:
import functools

def count_call(func):
    @functools.wraps(func)
    def wrapper_count_call(*args , **kwargs):
        wrapper_count_call.num_calls +=1 #stateful
        print(f'call{wrapper_count_call.num_calls} of {func.__name__!r}')
        return func(*args , **kwargs)
    wrapper_count_call.num_calls = 0
    return wrapper_count_call

In [28]:
@count_call
def saywhee():
    print('whee!')

In [29]:
saywhee()

1 of 'saywhee'
whee!


In [30]:
class Counter_call:
    def __init__(self , start=0):
        self.count = start
        
    def __call__(self):
        self.count+=1
        print(f'current count is {self.count}')

In [31]:
counter= Counter_call()

In [32]:
counter()

current count is 1


In [33]:
class Count_calls:
    def __init__(self , func):
        functools.update_wrapper(self ,func)
        self.func= func
        self.num_calls=0
        
    def __call__(self , *args , **kwargs):
        self.num_calls+=1
        print(f'call {self.num_calls} of {self.func.__name__!r}')
        return self.func(*args , **kwargs)
        

In [34]:
@Count_calls
def say_whee():
    print('whee!')

In [35]:
say_whee()

call 1 of 'say_whee'
whee!


# دکوریتور با ارگمان ورودی بصورت آپشنال

In [50]:
def slow_downn(_func=None , * , rate = 1):
    def decorator_slow_downn(func):
        @functools.wraps(func)
        def wrapper_slow_downn(*args, **kwargs):
            time.sleep(rate)
            return func(*args , **kwargs)
        return wrapper_slow_downn
    if _func is None:
        return decorator_slow_downn
    else:
        return decorator_slow_downn(_func)


@slow_downn(rate=5)
def count_downn(num):
    if num<=1:
        print(num)
        print ('liftoff')
    else:
        print(num)
        count_downn(num-1)

In [51]:
count_downn(3)

3
2
1
liftoff


# singletonیک کلاس با تنها یک نمونه است و نمی توان نمونه های بیشتری از آن ایجاد کرد
#Fale,True,Noneهر کدام یک singletonهستند

In [57]:
def singleton(cls):
    @functools.wraps(cls)
    def wrapper_singleton(*args , **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance=cls(*args , **kwargs)
            return wrapper_singleton.instance
        wrapper_singleton.instance = None
        return wrapper_singleton
    
    
@singleton
class TheOne:
    pass
            

In [59]:
first_one= TheOne

In [60]:
another_one = TheOne

In [61]:
id(first_one)

140712979098352

In [62]:
id(another_one)

140712979098352

In [63]:
first_one is another_one

True

# cachکردن مقادیر بازگشتی

In [14]:
import functools
def cach(func):
    @functools.wraps(func)
    def wrapper_cach(*args , **kwargs):
        cach_key = args + tuple(kwargs.items())
        if cach_key not in wrapper_cach.cach:
            wrapper_cach.cach[cach_key] = func(*args , **kwargs)
        return wrapper_cach.cach[cach_key]
    wrapper_cach.cach=dict()
    return wrapper_cach
    

    
@cach
@count_call
def fibonacci(num):
    if num<2:
        return num
    return fibonacci(num-1) + fibonacci(num-2)
    

In [15]:
fibonacci(21)

call1 of 'fibonacci'
call2 of 'fibonacci'
call3 of 'fibonacci'
call4 of 'fibonacci'
call5 of 'fibonacci'
call6 of 'fibonacci'
call7 of 'fibonacci'
call8 of 'fibonacci'
call9 of 'fibonacci'
call10 of 'fibonacci'
call11 of 'fibonacci'
call12 of 'fibonacci'
call13 of 'fibonacci'
call14 of 'fibonacci'
call15 of 'fibonacci'
call16 of 'fibonacci'
call17 of 'fibonacci'
call18 of 'fibonacci'
call19 of 'fibonacci'
call20 of 'fibonacci'
call21 of 'fibonacci'
call22 of 'fibonacci'


10946

In [16]:
fibonacci(8)

21

In [17]:
fibonacci(23)

call23 of 'fibonacci'
call24 of 'fibonacci'


28657

# cacheکردن مقادیر بازگشتی با cacheدر کتابخانه functools
@functools.lru_cache(Least Recently Used(LRU))

In [20]:
@count_call
@functools.lru_cache(maxsize=7)
def fibonacci(num):
    if num<2:
        return num
    return fibonacci(num-1)+fibonacci(num-2)

In [22]:
fibonacci(16)

call120 of 'fibonacci'
call121 of 'fibonacci'
call122 of 'fibonacci'
call123 of 'fibonacci'
call124 of 'fibonacci'
call125 of 'fibonacci'
call126 of 'fibonacci'
call127 of 'fibonacci'
call128 of 'fibonacci'
call129 of 'fibonacci'
call130 of 'fibonacci'
call131 of 'fibonacci'
call132 of 'fibonacci'
call133 of 'fibonacci'
call134 of 'fibonacci'
call135 of 'fibonacci'
call136 of 'fibonacci'
call137 of 'fibonacci'
call138 of 'fibonacci'
call139 of 'fibonacci'
call140 of 'fibonacci'
call141 of 'fibonacci'
call142 of 'fibonacci'
call143 of 'fibonacci'
call144 of 'fibonacci'
call145 of 'fibonacci'
call146 of 'fibonacci'
call147 of 'fibonacci'
call148 of 'fibonacci'
call149 of 'fibonacci'
call150 of 'fibonacci'


987

In [23]:
fibonacci(21)

call151 of 'fibonacci'
call152 of 'fibonacci'
call153 of 'fibonacci'
call154 of 'fibonacci'
call155 of 'fibonacci'
call156 of 'fibonacci'
call157 of 'fibonacci'
call158 of 'fibonacci'
call159 of 'fibonacci'
call160 of 'fibonacci'
call161 of 'fibonacci'


10946